In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels.stats.multitest as smm
import numpy as np
#from bioinfokit import analys, visuz
from matplotlib.patches import Circle
from matplotlib.patheffects import withStroke
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
import matplotlib

color_palette = sns.color_palette("colorblind")

sns.set_theme(palette=color_palette, font="Arial", font_scale=1.0, style="white")


#matplotlib.rcParams.update({'axes.labelsize': 16,'xtick.labelsize': 16,})
import os


In [2]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 25)
#pd.set_option('display.max_colwidth', 300)
#pd.set_option('format.precision', 2)




In [3]:
# load results table from R DESeq analysis (gene-level, LFC shrinkage)
results_genes=pd.read_csv("shrink_Anno_male_only_231212.csv")  
results_genes.rename(columns={"Unnamed: 0": "GENEID"}, inplace = True)

results_genes#.dropna(axis=0, subset=["SYMBOL"])
results_genes #all GENEIDs are unique

,GENEID,baseMean,log2FoldChange,lfcSE,pvalue,padj,GENENAME,SYMBOL,ENTREZID
0,ENSMUSG00000000001,5868.589363,0.208028,0.117759,0.002531,0.231199,Gnai3,Gnai3,14679.0
1,ENSMUSG00000000003,0.000000,NaN,NaN,NaN,NaN,Pbsn,Pbsn,54192.0
2,ENSMUSG00000000028,84.565653,0.002974,0.038890,0.561446,0.999632,Cdc45,Cdc45,12544.0
3,ENSMUSG00000000037,18.402719,-0.004037,0.039310,0.062883,0.880304,Scml2,Scml2,107815.0
4,ENSMUSG00000000049,0.480106,-0.000024,0.039055,0.969369,NaN,Apoh,Apoh,11818.0
...,...,...,...,...,...,...,...,...,...
35677,ENSMUSG00000118670,1.963773,-0.000399,0.039053,0.576224,NaN,NaN,NaN,NaN
35678,ENSMUSG00000118671,258.713465,-0.005450,0.039271,0.290983,0.999632,EPPK1,EPPK1,NaN
35679,ENSMUSG00000118672,0.000000,NaN,NaN,NaN,NaN,Muc4,Muc4,140474.0
35680,ENSMUSG00001074846,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,68265.0


In [4]:
#formatting
data=results_genes
data["padj_rounded"] = data["padj"].map('{:.1e}'.format)
data["GENENAME_padj"] = data["GENENAME"] + "\np=" + data["padj_rounded"].astype(str) +""
data["-log10_padj"] = -np.log10(data["padj"])

In [5]:
padj_thr = 0.05
lfc_thr = 1




data_all = data.dropna(subset=["log2FoldChange", "padj", "GENENAME"])
data_all_up = data_all[(data_all["log2FoldChange"] > lfc_thr) & (data_all["padj"] < 0.05)]
data_all_down = data_all[(data_all["log2FoldChange"] < -lfc_thr) & (data_all["padj"] < 0.05)]

In [6]:
data_DE_all_sortL2FC = pd.concat([data_all_up.sort_values(by=["log2FoldChange"], ascending=False),
           data_all_down.sort_values(by=["log2FoldChange"], ascending=False)])

print(len(data_DE_all_sortL2FC))
data_DE_all_sortL2FC

15


,GENEID,baseMean,log2FoldChange,lfcSE,pvalue,padj,GENENAME,SYMBOL,ENTREZID,padj_rounded,GENENAME_padj,-log10_padj
20283,ENSMUSG00000079224,46.828020,10.392317,2.899889,4.876740e-14,2.464542e-10,Gm6565,Gm6565,NaN,2.5e-10,Gm6565\np=2.5e-10,9.608264
20089,ENSMUSG00000078746,51.504298,10.108249,3.299635,2.558289e-07,3.526020e-04,Fam205a4,Fam205a4,545611.0,3.5e-04,Fam205a4\np=3.5e-04,3.452715
19020,ENSMUSG00000073878,6.617288,6.573520,2.843841,5.297349e-05,1.958856e-02,Gm13304,Gm13304,100504346.0,2.0e-02,Gm13304\np=2.0e-02,1.707997
10926,ENSMUSG00000038155,64.622530,3.453471,0.460577,5.989624e-15,4.540434e-11,Gstp2,Gstp2,14869.0,4.5e-11,Gstp2\np=4.5e-11,10.342903
15684,ENSMUSG00000055632,67.636318,1.913543,0.642410,1.112840e-04,3.374353e-02,Hmcn2,Hmcn2,665700.0,3.4e-02,Hmcn2\np=3.4e-02,1.471809
7695,ENSMUSG00000030270,34.724822,1.869879,0.649687,1.407971e-04,3.715592e-02,Cpne9,Cpne9,211232.0,3.7e-02,Cpne9\np=3.7e-02,1.429972
3089,ENSMUSG00000021453,236.026645,1.276542,0.205166,2.544934e-11,9.645935e-08,Gadd45g,Gadd45g,23882.0,9.6e-08,Gadd45g\np=9.6e-08,7.015656
5302,ENSMUSG00000026064,215.115427,1.224926,0.288309,1.030799e-06,1.034051e-03,Ptp4a1,Ptp4a1,19243.0,1.0e-03,Ptp4a1\np=1.0e-03,2.985458
12347,ENSMUSG00000042041,158.155309,1.168865,0.405033,1.445946e-04,3.715592e-02,2010003K11Rik,2010003K11Rik,69861.0,3.7e-02,2010003K11Rik\np=3.7e-02,1.429972
7566,ENSMUSG00000030056,1313.851273,1.068383,0.181176,1.914466e-10,5.805043e-07,Isy1,Isy1,57905.0,5.8e-07,Isy1\np=5.8e-07,6.236195


In [7]:
data_DE_all_sortL2FC.sort_values(by=["log2FoldChange"], ascending=True)[["GENENAME", 
                                                                         "log2FoldChange", 
                                                                         "padj_rounded"]]\
.round(3)\
.to_excel("data_l2fc_p_DEG_only_male_231212.xlsx")

In [8]:
genes_of_interest = ["Cacna1a",
                    "Cacna1b",
                    "Cacna1c",
                    "Cacna1d",
                    "Cacna1e",
                    "Cacna1f",
                    "Cacna1g",
                    "Cacna1h",
                    "Cacna1i",
                    "Cacna2d1",
                    "Cacna2d2",
                    "Cacnb1",
                    "Cacnb2",
                    "Cacnb3",
                    "Cacnb4"
                    ]

In [9]:
data_goi = data[data["GENENAME"].isin(genes_of_interest)]
data_goi.sort_values(by=["GENENAME"], ascending=True)

,GENEID,baseMean,log2FoldChange,lfcSE,pvalue,padj,GENENAME,SYMBOL,ENTREZID,padj_rounded,GENENAME_padj,-log10_padj
9696,ENSMUSG00000034656,63.427161,-0.001445,0.039004,0.582253,0.999632,Cacna1a,Cacna1a,12286.0,1.0e+00,Cacna1a\np=1.0e+00,0.000160
673,ENSMUSG00000004113,89.045965,-0.003663,0.039074,0.391144,0.999632,Cacna1b,Cacna1b,12287.0,1.0e+00,Cacna1b\np=1.0e+00,0.000160
14851,ENSMUSG00000051331,3734.249876,-0.003017,0.036963,0.807167,0.999632,Cacna1c,Cacna1c,12288.0,1.0e+00,Cacna1c\np=1.0e+00,0.000160
1674,ENSMUSG00000015968,487.065786,-0.007211,0.039070,0.391139,0.999632,Cacna1d,Cacna1d,12289.0,1.0e+00,Cacna1d\np=1.0e+00,0.000160
672,ENSMUSG00000004110,12.958336,-0.000831,0.039033,0.622237,0.999632,Cacna1e,Cacna1e,12290.0,1.0e+00,Cacna1e\np=1.0e+00,0.000160
8133,ENSMUSG00000031142,1.986726,0.000408,0.039053,0.580141,NaN,Cacna1f,Cacna1f,54652.0,nan,Cacna1f\np=nan,NaN
2768,ENSMUSG00000020866,27.214688,0.003225,0.039206,0.102713,0.956126,Cacna1g,Cacna1g,12291.0,9.6e-01,Cacna1g\np=9.6e-01,0.019485
4278,ENSMUSG00000024112,2702.579277,-0.005770,0.039616,0.029885,0.718494,Cacna1h,Cacna1h,58226.0,7.2e-01,Cacna1h\np=7.2e-01,0.143577
3640,ENSMUSG00000022416,24.844881,0.001035,0.039000,0.671701,0.999632,Cacna1i,Cacna1i,239556.0,1.0e+00,Cacna1i\np=1.0e+00,0.000160
11696,ENSMUSG00000040118,8187.157426,0.023205,0.047466,0.046128,0.815109,Cacna2d1,Cacna2d1,12293.0,8.2e-01,Cacna2d1\np=8.2e-01,0.088784


In [10]:
data_goi[["GENENAME", "log2FoldChange", "padj"]].to_excel("data_l2fc_p_calcium_channels_only_male_231212.xlsx")

In [11]:
kegg_list = pd.read_csv("Kegg_list_mmu.csv", 
                        usecols=['GeneID', 'PathwayID','Symbol', 'ENSEMBL', "Genename"])
kegg_aldo = kegg_list[kegg_list["PathwayID"]=="path:mmu04925"]

data_aldo_genes = kegg_aldo.merge(results_genes, how="left", left_on='ENSEMBL', right_on='GENEID', validate="1:1")
print(len(data_aldo_genes.sort_values(by=["log2FoldChange"], ascending=True)))
data_aldo_genes.sort_values(by=["log2FoldChange"], ascending=True)

102


,GeneID,PathwayID,Symbol,ENSEMBL,Genename,GENEID,baseMean,log2FoldChange,lfcSE,pvalue,padj,GENENAME,SYMBOL,ENTREZID,padj_rounded,GENENAME_padj,-log10_padj
77,223604,path:mmu04925,Kcnk9,ENSMUSG00000036760,"potassium channel, subfamily K, member 9",ENSMUSG00000036760,9189.902143,-0.048660,0.091233,0.012376,0.541681,Kcnk9,Kcnk9,223604.0,5.4e-01,Kcnk9\np=5.4e-01,0.266257
53,16835,path:mmu04925,Ldlr,ENSMUSG00000032193,low density lipoprotein receptor,ENSMUSG00000032193,2676.356273,-0.038893,0.066143,0.023972,0.673318,Ldlr,Ldlr,16835.0,6.7e-01,Ldlr\np=6.7e-01,0.171780
55,17200,path:mmu04925,Mc2r,ENSMUSG00000045569,melanocortin 2 receptor,ENSMUSG00000045569,4028.434724,-0.027558,0.050806,0.051820,0.837808,Mc2r,Mc2r,17200.0,8.4e-01,Mc2r\np=8.4e-01,0.076856
87,320707,path:mmu04925,Atp2b3,ENSMUSG00000031376,"ATPase, Ca++ transporting, plasma membrane 3",ENSMUSG00000031376,1896.452903,-0.026490,0.050692,0.039206,0.779627,Atp2b3,Atp2b3,320707.0,7.8e-01,Atp2b3\np=7.8e-01,0.108113
65,18795,path:mmu04925,Plcb1,ENSMUSG00000051177,"phospholipase C, beta 1",ENSMUSG00000051177,6479.631683,-0.024916,0.047862,0.176559,0.999632,Plcb1,Plcb1,18795.0,1.0e+00,Plcb1\np=1.0e+00,0.000160
67,18797,path:mmu04925,Plcb3,ENSMUSG00000024960,"phospholipase C, beta 3",ENSMUSG00000024960,1643.723950,-0.024415,0.049089,0.034160,0.757723,Plcb3,Plcb3,18797.0,7.6e-01,Plcb3\np=7.6e-01,0.120490
38,14672,path:mmu04925,Gna11,ENSMUSG00000034781,"guanine nucleotide binding protein, alpha 11",ENSMUSG00000034781,2008.067338,-0.019524,0.043332,0.142769,0.994588,Gna11,Gna11,14672.0,9.9e-01,Gna11\np=9.9e-01,0.002357
10,11515,path:mmu04925,Adcy9,ENSMUSG00000005580,adenylate cyclase 9,ENSMUSG00000005580,2286.524340,-0.017801,0.042066,0.184768,0.999632,Adcy9,Adcy9,11515.0,1.0e+00,Adcy9\np=1.0e+00,0.000160
35,12915,path:mmu04925,Atf6b,ENSMUSG00000015461,activating transcription factor 6 beta,ENSMUSG00000015461,586.311026,-0.016688,0.042792,0.102820,0.956126,Atf6b,Atf6b,12915.0,9.6e-01,Atf6b\np=9.6e-01,0.019485
39,14682,path:mmu04925,Gnaq,ENSMUSG00000024639,"guanine nucleotide binding protein, alpha q po...",ENSMUSG00000024639,1337.798013,-0.016270,0.041071,0.229534,0.999632,Gnaq,Gnaq,14682.0,1.0e+00,Gnaq\np=1.0e+00,0.000160


In [12]:
data_aldo_genes.sort_values(by=["padj", "log2FoldChange"], ascending=True).to_excel("kegg_aldo_mmu04925_genes_only_male_231212.xlsx",
                                                            columns=["Symbol", "log2FoldChange", "padj"])